In [2]:
import numpy as np
import pandas as pd
from multiprocessing import Pool
import matplotlib.pyplot as plt
import matplotlib.gridspec as grid_spec
from tqdm.auto import tqdm
from math import ceil
import itertools
import h5py
import io
import robustbench

In [15]:
dataset_path = "/data/output/20220226_robustness/dataset.h5"
df_meta = pd.read_hdf(dataset_path, "meta")
df_meta["filter_ids"] = df_meta["filter_ids"].apply(lambda s: np.arange(int(s.split(":")[0]), 1 + int(s.split(":")[1])))
df_meta.Robust = df_meta.Robust.apply(bool)

In [3]:
df_pairings = pd.read_csv("robust_vs_normal_pairings.csv", index_col=0)

In [4]:
df_pairings.Robust.nunique()

71

In [8]:
df_pairings.Arch.nunique()

13

In [10]:
df_pairings.Dataset.nunique()

3

In [14]:
all_used_models = set(list(df_pairings.Robust.unique()) + list(df_pairings.Normal.unique()))

In [17]:
df_meta[df_meta.Name.isin(df_pairings.Robust)].filter_ids.apply(len).sum()

615863744

#### Number of models

In [23]:
df_meta.index.get_level_values(0).unique().size

119

In [24]:
df_meta[df_meta.Robust].index.get_level_values(0).unique().size

73

In [25]:
df_meta[~df_meta.Robust].index.get_level_values(0).unique().size

46

In [36]:
df_meta.Name.unique()

array(['hso_normal_training_robustbench_Addepalli2021Towards_PARN18_cifar100_13',
       'hso_normal_training_robustbench_Rebuffi2021Fixing_R18_ddpm_cifar100_13',
       'hso_normal_training_robustbench_Rice2020Overfitting_cifar100_13',
       'hso_normal_training_robustbench_Addepalli2021Towards_RN18_cifar10_13',
       'hso_normal_training_robustbench_Andriushchenko2020Understanding_cifar10_13',
       'hso_normal_training_robustbench_Rebuffi2021Fixing_R18_ddpm_cifar10_13',
       'hso_normal_training_robustbench_Engstrom2019Robustness_cifar10_13',
       'hso_normal_training_robustbench_Ding2020MMA_cifar10_13',
       'hso_normal_training_robustbench_Hendrycks2019Using_cifar100_13',
       'hso_normal_training_robustbench_Hendrycks2019Using_cifar10_13',
       'hso_normal_training_robustbench_Cui2020Learnable_34_10_LBGAT0_cifar100_13',
       'hso_normal_training_robustbench_Chen2020Efficient_cifar100_13',
       'hso_normal_training_robustbench_Addepalli2021Towards_WRN34_cifar10_13

#### Number of depth (NOT layers!)

In [26]:
df_meta.index.size

3667

In [18]:
df_meta.groupby(["Robust", "Training-Dataset", "Backbone",]).Name.nunique()

Robust  Training-Dataset  Backbone               
False   cifar10           PreActResNet-18             3
                          ResNet-18                   2
                          ResNet-50                   2
                          WideResNet-106-16           1
                          WideResNet-28-10            3
                          WideResNet-28-10-PSSiLU     1
                          WideResNet-28-4             1
                          WideResNet-34-10            7
                          WideResNet-34-15            1
                          WideResNet-34-20            5
                          WideResNet-34-R             1
                          WideResNet-70-16            2
        cifar100          PreActResNet-18             3
                          WideResNet-28-10            2
                          WideResNet-34-10            4
                          WideResNet-34-20            1
                          WideResNet-70-16            

In [28]:
df_meta.groupby(["Robust", "Training-Dataset"]).Name.nunique()

Robust  Training-Dataset
False   cifar10             29
        cifar100            11
        imagenet             6
True    cifar10             49
        cifar100            17
        imagenet             5
Name: Name, dtype: int64

In [29]:
from robustbench.model_zoo import model_dicts as all_models
from robustbench.model_zoo.enums import BenchmarkDataset, ThreatModel
import inspect
from collections import defaultdict

paper_dict = defaultdict(lambda: defaultdict(list))
for dataset in BenchmarkDataset:
    for paper, info_dict in all_models[dataset][ThreatModel.Linf].items():
        paper_dict[dataset.value][inspect.getsource(info_dict["model"])].append(paper)

In [30]:
for dataset, models in paper_dict.items():
    for pair in models.values():
        print(dataset, pair, df_meta[(df_meta.Network.isin(pair)) & (df_meta["Training-Dataset"] == dataset)].Backbone.unique())

cifar10 ['Andriushchenko2020Understanding'] ['PreActResNet-18']
cifar10 ['Carmon2019Unlabeled', 'Sehwag2020Hydra', 'Wang2020Improving', 'Wu2020Adversarial_extra', 'Zhang2020Geometry', 'Sridhar2021Robust'] ['WideResNet-28-10']
cifar10 ['Hendrycks2019Using'] ['WideResNet-28-10']
cifar10 ['Rice2020Overfitting'] ['WideResNet-34-20']
cifar10 ['Zhang2019Theoretically', 'Huang2020Self', 'Zhang2019You', 'Zhang2020Attacks', 'Sitawarin2020Improving', 'Cui2020Learnable_34_10'] ['WideResNet-34-10']
cifar10 ['Engstrom2019Robustness'] ['ResNet-50']
cifar10 ['Chen2020Adversarial'] ['ResNet-50']
cifar10 ['Pang2020Boosting'] ['WideResNet-34-20']
cifar10 ['Wong2020Fast'] ['PreActResNet-18']
cifar10 ['Ding2020MMA'] ['WideResNet-28-4']
cifar10 ['Wu2020Adversarial'] ['WideResNet-34-10']
cifar10 ['Gowal2020Uncovering_70_16', 'Gowal2020Uncovering_70_16_extra', 'Rebuffi2021Fixing_70_16_cutmix_ddpm', 'Rebuffi2021Fixing_70_16_cutmix_extra'] ['WideResNet-70-16']
cifar10 ['Gowal2020Uncovering_34_20'] ['WideResNet

In [31]:
for dataset in BenchmarkDataset:
    for paper, info_dict in all_models[dataset][ThreatModel.Linf].items():
        if not paper in df_meta.Network.values:
            print(paper, dataset, "is missing")

Standard BenchmarkDataset.cifar_10 is missing
Kang2021Stable BenchmarkDataset.cifar_10 is missing
Standard_R50 BenchmarkDataset.imagenet is missing


In [32]:
paper_dict["imagenet"].values()

dict_values([['Wong2020Fast', 'Engstrom2019Robustness', 'Salman2020Do_R50'], ['Salman2020Do_R18'], ['Salman2020Do_50_2'], ['Standard_R50']])

In [43]:
pairings = []
for dataset in BenchmarkDataset:
    for paper, info_dict in all_models[dataset][ThreatModel.Linf].items():
        robust_model = df_meta[(df_meta.Network == paper) & (df_meta["Training-Dataset"] == dataset.value) & (df_meta.Robust)].Name.unique()
        
        if len(robust_model) < 1:
            print(paper, dataset.value)
            continue
        
        for pairs in paper_dict[dataset.value].values():
            if paper in pairs:
                counter_parts = pairs
        
        backbone = df_meta[(df_meta.Network.isin(counter_parts)) & (df_meta["Training-Dataset"] == dataset.value)].Backbone.unique()
                
        if dataset.value != "imagenet":
            normal_model = df_meta[(df_meta.Network.isin(counter_parts)) & (df_meta["Training-Dataset"] == dataset.value) & (~df_meta.Robust)].Name.unique()
        else:  # our imagenet models didnt train well, so let's use the ones from timm
            normal_model = df_meta[df_meta.Name.str.contains("timm_") & (df_meta.Backbone == backbone[0])].Name.unique()
        
       
        pairings.append((robust_model[0], normal_model[0], backbone[0], dataset.value))
        
df = pd.DataFrame(pairings, columns=["Robust", "Normal", "Arch", "Dataset"])
df.to_csv("robust_vs_normal_pairings.csv")
df

Standard cifar10
Kang2021Stable cifar10
Standard_R50 imagenet


,Robust,Normal,Arch,Dataset
0,robustbench_Andriushchenko2020Understanding_ci...,hso_normal_training_robustbench_Andriushchenko...,PreActResNet-18,cifar10
1,robustbench_Carmon2019Unlabeled_cifar10_linf_13,hso_normal_training_robustbench_Carmon2019Unla...,WideResNet-28-10,cifar10
2,robustbench_Sehwag2020Hydra_cifar10_linf_13,hso_normal_training_robustbench_Carmon2019Unla...,WideResNet-28-10,cifar10
3,robustbench_Wang2020Improving_cifar10_linf_13,hso_normal_training_robustbench_Carmon2019Unla...,WideResNet-28-10,cifar10
4,robustbench_Hendrycks2019Using_cifar10_linf_13,hso_normal_training_robustbench_Hendrycks2019U...,WideResNet-28-10,cifar10
...,...,...,...,...
66,robustbench_Wong2020Fast_imagenet_linf_13,timm_resnet50_imagenet1k_13,ResNet-50,imagenet
67,robustbench_Engstrom2019Robustness_imagenet_li...,timm_resnet50_imagenet1k_13,ResNet-50,imagenet
68,robustbench_Salman2020Do_R50_imagenet_linf_13,timm_resnet50_imagenet1k_13,ResNet-50,imagenet
69,robustbench_Salman2020Do_R18_imagenet_linf_13,timm_resnet18_imagenet1k_13,ResNet-18,imagenet


In [44]:
df_pairings["Paper"] =  df_pairings.Robust.apply(lambda name: df_meta[df_meta.Name == name].Network.values[0])
df_pairings["Adv. Trained Clean Acc"] =  df_pairings.Robust.apply(lambda name: df_meta[df_meta.Name == name]["Clean Accuracy"].values[0])
df_pairings["Adv. Trained Robust Acc"] = df_pairings.Robust.apply(lambda name: df_meta[df_meta.Name == name]["Robust Accuracy"].values[0])
df_pairings["Norm. Trained Clean Acc"] = df_pairings.Normal.apply(lambda name: df_meta[df_meta.Name == name]["Clean Accuracy"].values[0])
df_pairings["Norm. Trained Robust Acc"] = df_pairings.Normal.apply(lambda name: df_meta[df_meta.Name == name]["Robust Accuracy"].values[0])

print(df_pairings[["Paper", "Dataset", "Arch", "Adv. Trained Clean Acc", "Adv. Trained Robust Acc", "Norm. Trained Clean Acc", "Norm. Trained Robust Acc"]].to_latex(index=False, longtable=True))

\begin{longtable}{lllrrrr}
\toprule
                               Paper &  Dataset &                    Arch &  Adv. Trained Clean Acc &  Adv. Trained Robust Acc &  Norm. Trained Clean Acc &  Norm. Trained Robust Acc \\
\midrule
\endfirsthead

\toprule
                               Paper &  Dataset &                    Arch &  Adv. Trained Clean Acc &  Adv. Trained Robust Acc &  Norm. Trained Clean Acc &  Norm. Trained Robust Acc \\
\midrule
\endhead
\midrule
\multicolumn{7}{r}{{Continued on next page}} \\
\midrule
\endfoot

\bottomrule
\endlastfoot
     Andriushchenko2020Understanding &  cifar10 &         PreActResNet-18 &                   79.84 &                    43.93 &                    94.51 &                       0.0 \\
                 Carmon2019Unlabeled &  cifar10 &        WideResNet-28-10 &                   89.69 &                    59.53 &                    95.10 &                       0.0 \\
                     Sehwag2020Hydra &  cifar10 &        WideResNet-28-1